# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [36]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
import pickle

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

A potential problem statement would be to take in the message as input to the ML pipeline and classify it into categories as 'related', 'request' and so on. So, X or the independent variable should be 'message' and y should be all the classes.

In [37]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('df', con=engine)


In [38]:
#Dropping unnecessary columns
df=df.drop(["original","genre"],axis=1)
#dropping rows with null values
df=df.dropna(how='any')

In [39]:
df.head()

,id,message,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Write a tokenization function to process your text data

In [40]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [41]:
#As instructed, choosing he 'message' column as input and output classification results on the other 36 categories in the datas
X = df["message"]
y = df[df.columns[3:]] #Not taking the 'related' column as the value of that column can be determined by the other columns

# Create pipeline with Classifier
multiclass = MultiOutputClassifier(RandomForestClassifier())

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', multiclass)
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [42]:
# split data, train and predict
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [43]:
#Converting the prediction into a dataframe
y_pred=pd.DataFrame(y_pred)
y_pred.columns=y_test.columns

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [44]:
def display_results(y_test, y_pred):
    d = []
    for col in y_test.columns:
        category=col
        precision, recall, f_score, support = precision_recall_fscore_support(np.array(y_test[col]),np.array(y_pred[col]), average='weighted')
        d.append({'Category': category, 'Precision': precision, 'Recall': recall, 'f-Score': f_score})
    return pd.DataFrame(d)

In [45]:
display_results(y_test, y_pred)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Category,Precision,Recall,f-Score
0,request,0.765618,0.822039,0.772183
1,offer,0.988434,0.994200,0.991309
2,aid_related,0.536710,0.564866,0.530244
3,medical_help,0.857415,0.918498,0.885292
4,medical_products,0.911580,0.948565,0.929291
5,search_and_rescue,0.947846,0.971917,0.959730
6,security,0.966702,0.983059,0.974812
7,military,0.934513,0.965201,0.949610
8,child_alone,1.000000,1.000000,1.000000
9,water,0.888710,0.933913,0.907497


In [46]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fcb22c96400>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 6. Improve your model
Use grid search to find better parameters. 

In [47]:
parameters = {'clf__estimator__max_depth': [8, 40, None],
              'clf__estimator__min_samples_leaf':[4, 6, 14]}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [48]:
cv.fit(X_train, y_train)
y_pred_tuned = cv.predict(X_test)
y_pred_tuned = pd.DataFrame(y_pred_tuned)
y_pred_tuned.columns=y_test.columns
display_results(y_test, y_pred_tuned)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Category,Precision,Recall,f-Score
0,request,0.686581,0.828602,0.750936
1,offer,0.988434,0.994200,0.991309
2,aid_related,0.345065,0.587302,0.434716
3,medical_help,0.853762,0.923993,0.887490
4,medical_products,0.910521,0.954212,0.931855
5,search_and_rescue,0.947889,0.973596,0.960570
6,security,0.966704,0.983211,0.974888
7,military,0.934562,0.966728,0.950373
8,child_alone,1.000000,1.000000,1.000000
9,water,0.880194,0.938187,0.908266


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

I'm adding the StartingVerbExtractor feature from the tutorial and will try the desicion tree algorithm.

In [51]:
def tokenize_new(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize_new(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [52]:
nltk.pos_tag(tokenize(nltk.sent_tokenize(df["message"][0])[0]))[0]

('weather', 'NN')

In [53]:
pipeline_new = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', RandomForestClassifier())
])


In [54]:
pipeline_new.fit(X_train, y_train)
y_pred_dec_tree = pipeline_new.predict(X_test)

y_pred_dec_tree=pd.DataFrame(y_pred_dec_tree)
y_pred_dec_tree.columns=y_test.columns

display_results(y_test,y_pred_dec_tree)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Category,Precision,Recall,f-Score
0,request,0.763685,0.822039,0.770378
1,offer,0.988434,0.994200,0.991309
2,aid_related,0.529708,0.567613,0.512505
3,medical_help,0.853677,0.922772,0.886880
4,medical_products,0.910481,0.953297,0.931397
5,search_and_rescue,0.947881,0.973291,0.960418
6,security,0.966702,0.983059,0.974812
7,military,0.934553,0.966422,0.950220
8,child_alone,1.000000,1.000000,1.000000
9,water,0.880141,0.937271,0.907808


### 9. Export your model as a pickle file

In [50]:
pickle.dump(cv, open('final_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.